# L5: Automate Event Planning

In this lesson, you will learn more about Tasks.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [2]:
from crewai import Agent, Crew, Task

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [3]:
import os
from utils import get_openai_api_key,get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [4]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

# Initialize the tools
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [5]:
# Agent 1: Venue Coordinator
venue_coordinator = Agent(
    role="Venue Coordinator",
    goal="Identify and book an appropriate venue "
    "based on event requirements",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "With a keen sense of space and "
        "understanding of event logistics, "
        "you excel at finding and securing "
        "the perfect venue that fits the event's theme, "
        "size, and budget constraints."
    )
)

In [6]:
 # Agent 2: Logistics Manager
logistics_manager = Agent(
    role='Logistics Manager',
    goal=(
        "Manage all logistics for the event "
        "including catering and equipmen"
    ),
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Organized and detail-oriented, "
        "you ensure that every logistical aspect of the event "
        "from catering to equipment setup "
        "is flawlessly executed to create a seamless experience."
    )
)

In [7]:
# Agent 3: Marketing and Communications Agent
marketing_communications_agent = Agent(
    role="Marketing and Communications Agent",
    goal="Effectively market the event and "
         "communicate with participants",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Creative and communicative, "
        "you craft compelling messages and "
        "engage with potential attendees "
        "to maximize event exposure and participation."
    )
)

## Creating Venue Pydantic Object

- Create a class `VenueDetails` using [Pydantic BaseModel](https://docs.pydantic.dev/latest/api/base_model/).
- Agents will populate this object with information about different venues by creating different instances of it.

In [8]:
from pydantic import BaseModel
# Define a Pydantic model for venue details 
# (demonstrating Output as Pydantic)
class VenueDetails(BaseModel):
    name: str
    address: str
    capacity: int
    booking_status: str

## Creating Tasks
- By using `output_json`, you can specify the structure of the output you want.
- By using `output_file`, you can get your output in a file.
- By setting `human_input=True`, the task will ask for human feedback (whether you like the results or not) before finalising it.

In [9]:
venue_task = Task(
    description="Find a venue in {event_city} "
                "that meets criteria for {event_topic}.",
    expected_output="All the details of a specifically chosen"
                    "venue you found to accommodate the event.",
    human_input=True,
    output_json=VenueDetails,
    output_file="venue_details.json",  
      # Outputs the venue details as a JSON file
    agent=venue_coordinator
)

- By setting `async_execution=True`, it means the task can run in parallel with the tasks which come after it.

In [10]:
logistics_task = Task(
    description="Coordinate catering and "
                 "equipment for an event "
                 "with {expected_participants} participants "
                 "on {tentative_date}.",
    expected_output="Confirmation of all logistics arrangements "
                    "including catering and equipment setup.",
    human_input=True,
    async_execution=True,
    agent=logistics_manager
)

In [11]:
marketing_task = Task(
    description="Promote the {event_topic} "
                "aiming to engage at least"
                "{expected_participants} potential attendees.",
    expected_output="Report on marketing activities "
                    "and attendee engagement formatted as markdown.",
    async_execution=True,
    output_file="marketing_report.md",  # Outputs the report as a text file
    agent=marketing_communications_agent
)

## Creating the Crew

**Note**: Since you set `async_execution=True` for `logistics_task` and `marketing_task` tasks, now the order for them does not matter in the `tasks` list.

In [12]:
# Define the crew with agents and tasks
event_management_crew = Crew(
    agents=[venue_coordinator, 
            logistics_manager, 
            marketing_communications_agent],
    
    tasks=[venue_task, 
           logistics_task, 
           marketing_task],
    
    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [13]:
event_details = {
    'event_topic': "Tech Innovation Conference",
    'event_description': "A gathering of tech innovators "
                         "and industry leaders "
                         "to explore future technologies.",
    'event_city': "San Francisco",
    'tentative_date': "2024-09-15",
    'expected_participants': 500,
    'budget': 20000,
    'venue_type': "Conference Hall"
}

**Note 1**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

**Note 2**: 
- Since you set `human_input=True` for some tasks, the execution will ask for your input before it finishes running.
- When it asks for feedback, use your mouse pointer to first click in the text box before typing anything.

In [14]:
result = event_management_crew.kickoff(inputs=event_details)

 [DEBUG]: == Working Agent: Venue Coordinator
 [INFO]: == Starting Task: Find a venue in San Francisco that meets criteria for Tech Innovation Conference.


> Entering new CrewAgentExecutor chain...
I need to find a venue in San Francisco that meets the criteria for the Tech Innovation Conference. I must consider the theme, size, and budget constraints for the event.

Action: Search the internet
Action Input: {"search_query": "tech conference venues in San Francisco"} 


Search results: Title: Discover Tech Conferences Events & Activities in San Francisco, CA
Link: https://www.eventbrite.com/d/ca--san-francisco/tech-conferences/
Snippet: Tech conferences events in San Francisco, CA · Tech Networking SF · Tech Networking SF · Tech Weekend by Founders Village | Dec 2024 · Tech Weekend by Founders ...
---
Title: Top 12 tech conferences in San Francisco for 2024 - TravelPerk
Link: https://www.travelperk.com/blog/top-tech-conferences-in-san-francisco/
Snippet: With over 200 speakers, the Sa

Thought: I need to explore the venues mentioned in the search results to find the best fit for the Tech Innovation Conference.
Action: Read website content
Action Input: {"website_url": "https://techsposv.com/san-francisco-tech-conferences/"} 

San Francisco Tech Conferences
Event Starts in: Hurry Up!
Add to Calendar
06/02/2022 9:00 AM
06/03/2022 4:00 PM
Pacific
TECHSPO Silicon Valley 2022
Where Business, Tech and Innovation Collide in Silicon Valley!
TECHSPO Silicon Valley 2022 is a two day technology expo taking place June 2nd to 3rd, 2022 at Airport Marriott Waterfront in San Francisco, California in the heart of Silicon Beach. TECHSPO LA brings together developers, brands, marketers, technology providers, designers, innovators and evangelists looking to set the pace in our advanced world of technology. Exhibitors showcase the next generation of technology & innovation, including; Internet, Mobile, Adtech, Martech and SaaS technologies. Be prepared to be inspired, amazed and educate

Thought: Based on the information gathered from the website content, I need to focus on the TECHSPO Silicon Valley event at the Airport Marriott Waterfront in San Francisco.

Action: Read website content
Action Input: {"website_url": "https://techsposv.com/san-francisco-tech-conferences/"} 

I tried reusing the same input, I must stop using this action input. I'll try something else instead.



Action: Read website content
Action Input: {"website_url": "https://techsposv.com"} 

TECHSPO Silicon Valley 2024 · Technology Expo · July 8 - 9, 2024 · (Internet ~ Mobile ~ AdTech ~ MarTech ~ SaaS)
Event Starts in: Hurry Up!
Add to Calendar
06/02/2022 9:00 AM
06/03/2022 4:00 PM
Pacific
TECHSPO Silicon Valley 2022
Where Business, Tech and Innovation Collide in Silicon Valley!
TECHSPO Silicon Valley 2022 is a two day technology expo taking place June 2nd to 3rd, 2022 at Airport Marriott Waterfront in San Francisco, California in the heart of Silicon Beach. TECHSPO LA brings together developers, b

This is the agent final answer: TECHSPO Silicon Valley 2024 is a two-day technology expo taking place on July 8-9, 2024, at the Airport Marriott Waterfront in San Francisco. The event showcases the next generation of technology and innovation in areas such as Internet, Mobile, AdTech, MarTech, and SaaS technologies. Attendees will have the opportunity to interact with technology enthusiasts, network with industry professionals, and learn about the latest advancements impacting businesses for greater growth. The event also includes limited attendance conferences, such as DigiMarCon Silicon Valley Digital Marketing Conference. The expo features different event zones like TECHSPO Hall, Training Theater, Podcast Studio, Dining Hall, and DigiMarCon Auditorium. Various ticket options are available for visitors to choose their preferred experience, including Visitor Pass, Training Pass, and All Access Pass. Attendees can register for their passes online and secure their spot at the event.
Ple

Thought: I have gathered information about catering and equipment setup services for events in San Francisco.

Action: Delegate work to co-worker
Action Input: {"coworker": "Venue Coordinator", "task": "Contact catering and equipment rental companies for availability and quotes", "context": "I have gathered information about catering and equipment setup services for events in San Francisco. We need to coordinate catering and equipment setup for the event at the Airport Marriott Waterfront on 2024-09-15. The venue has a capacity of 1000 and is located in San Francisco."}

> Entering new CrewAgentExecutor chain...
Thought: I have gathered information on potential marketing activities and attendee engagement for the Tech Innovation Conference.

Final Answer:
# Marketing Activities and Attendee Engagement Report

## Marketing Activities:
- Incorporating interactive technology can significantly enhance engagement by making the event more dynamic and providing real-time feedback.
- Utilizing

Exception in thread Thread-5 (_execute):
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/usr/local/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/usr/local/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.11/site-packages/crewai/task.py", line 182, in _execute
    result = agent.execute_task(
             ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/site-packages/crewai/agent.py", line 221, in execute_task
    result = self.agent_executor.invoke(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/site-packages/langchain/chains/base.py", line 163, in invoke
    raise e
  File "/usr/local/lib/python3.11/site-packages/langchain/chains/base.py", line 153, in invoke
    self._call(inputs, run_manager=run_manager)
  File "/us

- Display the generated `venue_details.json` file.

In [15]:
import json
from pprint import pprint

with open('venue_details.json') as f:
   data = json.load(f)

pprint(data)

{'address': 'San Francisco',
 'booking_status': 'Available',
 'capacity': 1000,
 'name': 'Airport Marriott Waterfront'}


- Display the generated `marketing_report.md` file.

**Note**: After `kickoff` execution has successfully ran, wait an extra 45 seconds for the `marketing_report.md` file to be generated. If you try to run the code below before the file has been generated, your output would look like:

```
marketing_report.md
```

If you see this output, wait some more and than try again.

In [16]:
from IPython.display import Markdown
Markdown("marketing_report.md")

# Marketing Activities and Attendee Engagement Report

## Marketing Activities:
- Incorporating interactive technology can significantly enhance engagement by making the event more dynamic and providing real-time feedback.
- Utilizing event technology as a catalyst for reshaping the event industry and crafting unforgettable experiences.
- Driving event success with attendee engagement technology.
- Leveraging tech integration to boost engagement at corporate events.
- Implementing tech-enhanced event storytelling for enhanced attendee engagement.

## Attendee Engagement Strategies:
- Interactive workshops to captivate the audience.
- Live polling and Q&A sessions.
- Gamification elements for increased engagement.
- Networking opportunities for attendees.
- Utilizing interactive technology throughout the event.
- Engaging speakers to provide valuable insights.
- Educating exhibitors on good booth design and interactive activities.
- Leveraging social media outreach to maximize engagement.
- Track engagement metrics and analyze social media sentiment for post-event feedback.

By implementing these marketing activities and attendee engagement strategies, the Tech Innovation Conference aims to engage at least 500 potential attendees effectively.